In [1]:
import pandas as pd
import numpy as np
import torch
import faiss
import pickle
from sentence_transformers import SentenceTransformer

!pip freeze > requirements.txt

c:\Users\Intel7\Desktop\MAS\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model_save_path = r'C:\Users\Intel7\Desktop\MAS\all-MiniLM-L6-v2'
model = SentenceTransformer(model_save_path)

In [3]:
df = pd.read_csv(r'C:\Users\Intel7\Desktop\MAS\Dataset\data.csv', encoding="ISO-8859-1")

df['OriginalDescription'] = df['Description']

# Data Cleaning
df.dropna(subset=['Description'], inplace=True)  # Remove rows with missing descriptions
df['Description'] = df['Description'].str.lower()  # Convert to lowercase
df['Description'] = df['Description'].str.replace(r'[^a-zA-Z0-9 ]', '', regex=True)  # Remove special characters
# elimination of NaN values
df.dropna(inplace=True)
# elimination of duplicate rows
df.drop_duplicates(inplace=True)
# elimination of cancelled orders
df = df[~df['InvoiceNo'].str.startswith('C')]

# remove duplicate descriptions 
df.drop_duplicates(subset=['Description'], inplace=True, ignore_index=True)
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,OriginalDescription
0,536365,85123A,white hanging heart tlight holder,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,WHITE HANGING HEART T-LIGHT HOLDER
1,536365,71053,white metal lantern,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,WHITE METAL LANTERN
2,536365,84406B,cream cupid hearts coat hanger,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,CREAM CUPID HEARTS COAT HANGER
3,536365,84029G,knitted union flag hot water bottle,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,KNITTED UNION FLAG HOT WATER BOTTLE
4,536365,84029E,red woolly hottie white heart,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,RED WOOLLY HOTTIE WHITE HEART.


In [30]:
# Encode the product descriptions
descriptions = df['Description'].tolist()
encoded_descriptions = model.encode(descriptions)

# Create a Faiss index
dim = encoded_descriptions.shape[1]
index = faiss.IndexFlatL2(dim)

# Add the encoded descriptions to the Faiss index
index.add(encoded_descriptions)

In [31]:
def search_faiss(query, model, index, k=5):
    query = model.encode([query])[0]
    query = np.expand_dims(query, axis=0)
    Distances, Indices = index.search(query, k)
    return Distances, Indices

# Search for similar products
query = "big mug"
Distances, Indices = search_faiss(query, model, index)

# Print the similar products
for i in range(len(Indices[0])):
    print(Indices[0][i])
    print(df['OriginalDescription'].iloc[Indices[0][i]])
    print('Distance:', Distances[0][i])
    print()

2871
BIG POLKADOT MUG
Distance: 0.45231256

1065
GLAMOROUS  MUG
Distance: 0.52221566

1478
POTTERING MUG
Distance: 0.6258617

925
HOME SWEET HOME MUG
Distance: 0.6635146

1160
LOCAL CAFE MUG
Distance: 0.6828939



In [32]:
from agno.agent import Agent

class SearchAgent(Agent):
    def __init__(self, index, model, df):
        super().__init__()
        self.index = index
        self.model = model
        self.df = df

    def search(self, query, k=5):
        encoded_query = self.model.encode([query])
        distances, indices = self.index.search(encoded_query, k)
        results = [self.df['OriginalDescription'].iloc[indices[0][i]] for i in range(len(indices[0]))]
        return results

# Initialize the search agent
search_agent = SearchAgent(index=index, model=model, df=df)

# Example usage
query = "Phone"
results = search_agent.search(query)
print("Search results:")
for result in results:
    print(result)

Search results:
CRYSTAL FROG PHONE CHARM
CRYSTAL STILETTO PHONE CHARM
CRYSTAL SEA HORSE PHONE CHARM
PINK CRYSTAL SKULL PHONE CHARM
PINK CRYSTAL GUITAR PHONE CHARM


## Agents

In [37]:
from agno.agent import Agent, RunResponse
from agno.models.ollama import Ollama
from agno.tools.duckduckgo import DuckDuckGoTools


agent = Agent(
    model=Ollama(id="llama3:latest"),
    markdown=False,
    description="You are an math expert!",
)

response = agent.run("Whats happening in France?")
print(response.content)

Bonjour!

As a math expert, I don't have much to say about the latest French news or politics. But if you're interested in some interesting mathematical facts or connections related to France, I'd be happy to share some tidbits!

For instance, did you know that the famous mathematician Blaise Pascal was born in Clermont-Ferrand, France? He's known for his work on probability theory and was a pioneer in the field of calculus.

Or how about this: The French mathematician Pierre-Simon Laplace is credited with developing the nebular hypothesis, which proposes that stars and planets form from the condensation of interstellar gas. His work laid the foundation for modern astrophysics!

If you're interested in more math-related fun facts or connections to France, I'd be happy to share!


In [49]:
from agno.agent import Agent, RunResponse
from agno.models.huggingface import HuggingFace
from agno.tools.googlesearch import GoogleSearchTools
from agno.tools.wikipedia import WikipediaTools
from agno.tools.yfinance import YFinanceTools
import os

HF_token = 'API_TOKEN'
os.environ['HF_TOKEN'] = HF_token

agent = Agent(
    model=HuggingFace(
        id="meta-llama/Meta-Llama-3-8B-Instruct",
        max_tokens=4096,
    ),
    markdown=True, 
    tools = [WikipediaTools(), YFinanceTools(), GoogleSearchTools()],
    show_tool_calls=True,
    description="You are an AI assistant!"
)

response = agent.run("What is today's date?")
print(response.content)

response = agent.run("Search on wikipedia: Who is currently the president of the United States?")
print(response.content)

response = agent.run("Get the current stock price and recent history for AAPL")
print(response.content)

What is today's date?


ERROR    Unable to decode function arguments:                                                                      
         {'query': 'president_of_the_united_states'}                                                               
         Error: the JSON object must be str, bytes or bytearray, not dict

HfHubHTTPError: 422 Client Error: Unprocessable Entity for url: https://api-inference.huggingface.co/models/meta-llama/Meta-Llama-3-8B-Instruct/v1/chat/completions (Request ID: Root=1-67c49f39-1aee8f564940d0577053ede2;e387f110-b291-467a-96b3-a4ced2aa83f8)

Failed to deserialize the JSON body into the target type: messages[2].content: data did not match any variant of untagged enum MessageContent at line 1 column 1684
Make sure 'text-generation' task is supported by the model.